In this task we have to test your KNN Classifier on a kaggle competition. 

- The first competition is [Otto Group Product Classification Challenge](http://www.kaggle.com/c/otto-group-product-classification-challenge), here the goal is to place the given object in its right category.

In [7]:
%pylab inline
import scipy.stats as ss
import scipy.spatial.distance as ssd
from collections import defaultdict  # default dictionary 
from collections import Counter
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [8]:
import dmStyle
dmStyle.customize_mpl()


Setting custom matplotlib visual style


In [9]:
import time

class KNearestNeighbor:
    
    def __init__(self, k, scalefeatures=False):        
        self.k=k
        self.scalefeatures=scalefeatures
        self.frange=[-3, 3]
        
    def train(self, XTrain, Y,XTest,withargpartation):
       
        
        nexamples,nfeatures=XTrain.shape
        ## now go and train a model for each class...
        if self.scalefeatures:
            XTrain=self.scale_features(XTrain)        
            self.scalefeatures=False
        #Your code goes here...

        predictedLables=[]
        for i in range(len(XTest)):
            lables=[]
            dist=np.sqrt(np.sum((XTrain-XTest[i])**2,axis=-1))
            if withargpartation==True:
                k=np.argpartition(dist,self.k)
                k=k[:self.k]
                for i in k:
                    lables.append(Y[i])
                counter=Counter(lables)
                lable=np.argmax(counter)
                predictedLables.append(lables[lable])
            else:
                d=np.array(dist)
                d.sort()
                for i in range(self.k):
                    lables.append(Y[i])
                counter=Counter(lables)
                lable=np.argmax(counter)
                predictedLables.append(lables[lable])
        return predictedLables
    
    def predict(self, X, Y):
        
        if self.scalefeatures:
            X=(X-self.xmin)/(self.xmax-self.xmin)
        
        Ypred = []

        #your code goes here...
    
                
    def scale_features(self,X):
        self.xmin= np.min(X,axis=0)
        self.xmax= np.max(X,axis=0)

        return (X-self.xmin)/(self.xmax-self.xmin)


In [10]:
def class_2_matrix(Y, nclasses):
    
    ntest=len(Y)
    pclassm=np.zeros((ntest,nclasses))

    for i in range(len(Y)):
        j=int(Y[i].split('_')[1])-1
        pclassm[i,j]=1
    
    return pclassm

In [11]:
def logloss(plabels,olabels):
    
    nclasses=len(np.unique(olabels))
    
    olabels=class_2_matrix(olabels,nclasses)
    plabels=class_2_matrix(plabels,nclasses)
    
    epsilon = 1e-15
    import scipy as sp
    plabels = sp.maximum(epsilon, plabels) # clip the plabels
    plabels = sp.minimum(1-epsilon, plabels)
    logloss=-1.0/len(Ytest)*np.sum(np.log(plabels)*olabels)
    
    return logloss

In [12]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

## Otto Group Product Classification Challenge

In [13]:
data=pd.read_csv('./kaggle 2/train.csv')
print data.describe()
print data.columns

                 id       feat_1        feat_2        feat_3        feat_4  \
count  61878.000000  61878.00000  61878.000000  61878.000000  61878.000000   
mean   30939.500000      0.38668      0.263066      0.901467      0.779081   
std    17862.784315      1.52533      1.252073      2.934818      2.788005   
min        1.000000      0.00000      0.000000      0.000000      0.000000   
25%    15470.250000      0.00000      0.000000      0.000000      0.000000   
50%    30939.500000      0.00000      0.000000      0.000000      0.000000   
75%    46408.750000      0.00000      0.000000      0.000000      0.000000   
max    61878.000000     61.00000     51.000000     64.000000     70.000000   

             feat_5        feat_6        feat_7        feat_8        feat_9  \
count  61878.000000  61878.000000  61878.000000  61878.000000  61878.000000   
mean       0.071043      0.025696      0.193704      0.662433      1.011296   
std        0.438902      0.215333      1.030102      2.25577

In [14]:
Y=data['target']

In [15]:
#Seeing is beleaving, so lets start...
#Lets first plot the frequency of each class...
freq=scipy.stats.itemfreq(Y)# Get frequency of each class....
plt.bar(np.arange(1,len(freq[:,1])+1),freq[:,1],width=0.35)
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.title('Frequency of different class examples')

NameError: name 'scipy' is not defined

##Problem with Accuracy as evaluation Metric
Accuracy as evaluation metric works well when we have same number of examples for different classes, but for the datasets with imbalanced number of examples for each class accuracy is not preferred. For example consider you have dataset with 100 examples (95 from class-1 and 5 from class-2). Now having accuracy of 95% can mean correct classification of all the examples of class-1 with all examples of class-2 being miss-classified, which is not acceptable. So remember to never use the accuracy as evaluation metric for dataset with different number of examples for each class.

In [16]:
# Get your data in matrix
X=np.asarray(data.ix[:,1:-1].dropna(),dtype=np.float32)
print X.shape
Y=np.asarray(data.ix[:,-1])
print Y, Y.shape, len(np.unique(Y)) # so we have 9 classes...


(61878L, 93L)
['Class_1' 'Class_1' 'Class_1' ..., 'Class_9' 'Class_9' 'Class_9'] (61878L,) 9


In [54]:
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print " Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   
print " Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape   


 Training Data Set Dimensions= (43315L, 93L) Training True Class labels dimensions (43315L,)
 Test Data Set Dimensions= (18563L, 93L) Test True Class labels dimensions (18563L,)


In [133]:
# Lets train a Decision Tree Classifier on Petal Length and Width
feat=[0,1]
knn=KNearestNeighbor(3) # train a 3-nearest neighbour classifier...
k_nn=knn.train(Xtrain[:,feat],Ytrain,Xtest[:,feat],True)
print "DONE"

DONE


In [134]:
print k_nn

['Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_2', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_9', 'Class_2', 'Class_6', 'Class_6', 'Class_2', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_9', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_6', 'Class_2', 'Class_6', 'Class_6'

In [20]:
def write_in_kaggle_format(filename,ids,pclasses,nclasses=9):
    
    pclasses=class_2_matrix(pclasses,nclasses)
    
    dic=defaultdict(list) #build dictionary...
    dic['id']=ids
    for i in range(pclasses.shape[1]):
        dic['Class_'+str(i+1)]=pclasses[:,i]

    output = pd.DataFrame(dic,columns=['id','Class_1','Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])

    # Use pandas to write the comma-separated output file
    output.to_csv( filename, index=False, quoting=3 )

In [139]:
# Lets train a KNN on all the features... will take around 1 hour
feat=np.arange(Xtrain.shape[1])
knn=KNearestNeighbor(100)# With K as a random guess...
pclasses=knn.train(Xtrain[:,feat],Ytrain,Xtest[:,feat],True)
#pclasses=knn.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the logloss on the test set..
print "LogLoss = ", logloss(pclasses,Ytest)

LogLoss =  11.4837756779


In [140]:
print "LogLoss = ", logloss(pclasses,Ytest)

LogLoss =  11.4837756779


#Training on original training and kaggle test Set

In [17]:

datatest=pd.read_csv('./kaggle 2/test.csv')
ids=np.array(datatest['id'])
Xtest=np.array(datatest.ix[:,1:].values,dtype=np.float32)

In [14]:
#Lets train on 10 randomly selected features 
feat=np.arange(X.shape[1])
np.random.shuffle(feat)#using only ten features X.shape[1])
feat=feat[:10]
print feat
knn=KNearestNeighbor(5)# With K as a random guess...
pclasses=knn.train(X[:,feat],Y,Xtest[:,feat],True)
#pclasses=knn.predict(Xtest[:,feat])

[43 74 30 77 69  4 71 10 14 40]


KeyboardInterrupt: 

In [145]:
#Write in Kaggle Format...
write_in_kaggle_format('kaggel-otto-5features-100-nn',ids,pclasses)

